In [1]:
import numpy as np
import keras
from keras.layers import Dense # ImageNet
from tensorflow import random

In [ ]:
# Initiate the VGG16 model:

In [2]:
vgg_model = keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 1780s 3us/step


In [3]:
# Check the model summary

In [4]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
# Remove the last layer and Add all of them to the new model, except for the last layer

In [5]:
last_layer = str(vgg_model.layers[-1])
np.random.seed(42)
random.set_seed(42)

classifier= keras.Sequential()

for layer in vgg_model.layers:
    if str(layer) != last_layer:
        classifier.add(layer)

In [6]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [7]:
# Freezing the layers

In [8]:
for layer in classifier.layers:
    layer.trainable=False

In [9]:
# Add a new output layer of size 1

In [10]:
classifier.add(Dense(1, activation='sigmoid'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [11]:
# Compile the network

In [17]:
classifier.compile(optimizer='adam', loss='binary_crossentropy',
metrics=['accuracy'])

In [18]:
from keras.preprocessing.image import ImageDataGenerator
generate_train_data = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

generate_test_data = ImageDataGenerator(rescale =1./255)

In [19]:
training_dataset = generate_train_data.flow_from_directory('pneu/chest_xray/train',
                                             target_size = (224, 224),
                                             batch_size = 32,
                                             class_mode = 'binary')
test_datasetset = generate_test_data.flow_from_directory('pneu/chest_xray/val',
                                        target_size = (224, 224),
                                        batch_size = 32, class_mode = 'binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [20]:
# fit the model to the training data

In [16]:
classifier.fit_generator(training_dataset,
                         steps_per_epoch = 100,
                         epochs = 10,
                         validation_data = test_datasetset,
                         validation_steps = 30,
                         shuffle=False)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
100/100 [==============================] - 702s 7s/step - loss: 0.4157 - accuracy: 0.7975 - val_loss: 0.6566 - val_accuracy: 0.5625
Epoch 2/10
100/100 [==============================] - 715s 7s/step - loss: 0.2988 - accuracy: 0.8769
Epoch 3/10
100/100 [==============================] - 728s 7s/step - loss: 0.2862 - accuracy: 0.8781
Epoch 4/10
100/100 [==============================] - 686s 7s/step - loss: 0.2540 - accuracy: 0.8984
Epoch 5/10
100/100 [==============================] - 717s 7s/step - loss: 0.2364 - accuracy: 0.9041
Epoch 6/10
100/100 [==============================] - 719s 7s/step - loss: 0.2256 - accuracy: 0.9106
Epoch 7/10
100/100 [==============================] - 713s 7s/step - loss: 0.2183 - accuracy: 0.9150
Epoch 8/10
100/100 [==============================] - 2975s 30s/step - loss: 0.2146 - accuracy: 0.9112
Epoch 9/10
100/100 [==============================] - 628s 6s/step - los

In [ ]:
# Making the prediction using the predict method of the classifier.

In [ ]:
from keras.preprocessing import image

In [ ]:
new_image = image.load_img('test_image_2.jpg',
target_size = (224, 224))

new_image = image.img_to_array(new_image)
new_image = np.expand_dims(new_image, axis = 0)

result = classifier.predict(new_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'PNEUMONIA'
else:
    prediction = 'Normal'
print(prediction)